In [1]:
import numpy as np
import pandas as pd

### Prediction

In [2]:
flow_pred = np.load('./results/20230713_033341_long_term_forecast_flow_dm256_df256_el2_topk5_nk6_fq_h_Exp_3/pred.npy')
speed_pred = np.load('./results/20230713_033319_long_term_forecast_speed_dm256_df256_el2_topk5_nk6_fq_h_Exp_3/pred.npy')

In [3]:
D, L, K = flow_pred.shape
flow_pred = flow_pred.reshape(D * L, K) # (D*L,K)
speed_pred = speed_pred.reshape(D * L, K) # (D*L,K)

In [4]:
# flow_pred_df = pd.DataFrame(data=flow_pred, index=datetime_5min_test_ids_full)
# speed_pred_df = pd.DataFrame(data=speed_pred, index=datetime_5min_test_ids_full)

# pred_time_range
pred_ranges_5min = [('08:00:00', '08:55:00'), ('12:30:00', '13:25:00'), ('17:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in pred_ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

datetime_5min_test_ids = pd.DatetimeIndex(index_values)

flow_pred_df = pd.DataFrame(data=flow_pred, index=datetime_5min_test_ids)
speed_pred_df = pd.DataFrame(data=speed_pred, index=datetime_5min_test_ids)

In [5]:
result_flow_pred = np.sum(flow_pred_df.values.reshape((252,10,4)), axis=2)
result_speed_pred = np.round(np.sum((flow_pred_df.values * speed_pred_df.values).reshape((252,10,4)), axis=2) / result_flow_pred, decimals=2)


template_df = pd.read_csv('./submit_template.csv',index_col=['Time', 'Week'])

num_columns = result_flow_pred.shape[1]


result_df = pd.DataFrame(columns=range(num_columns * 2), index=template_df.index)


for i in range(num_columns):
    result_df.iloc[:, 2 * i] = result_flow_pred[:, i]
    result_df.iloc[:, 2 * i + 1] = result_speed_pred[:, i]

result_df.columns = template_df.columns

num_columns = result_df.shape[1]

for i in range(num_columns):
    if i % 2 == 0:
        result_df.iloc[:, i] = result_df.iloc[:, i].round(0).astype(int)

C:\Users\great\AppData\Local\Temp\ipykernel_26092\580094491.py:14: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  result_df.iloc[:, 2 * i] = result_flow_pred[:, i]
C:\Users\great\AppData\Local\Temp\ipykernel_26092\580094491.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  result_df.iloc[:, 2 * i + 1] = result_speed_pred[:, i]
C:\Users\great\AppData\Local\Temp\ipykernel_26092\580094491.py:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To 

In [6]:
result_df.head(10)

,,Loop1 Flow (Veh/h),Loop1 Speed (km/h),Loop2 Flow (Veh/h),Loop2 Speed (km/h),Loop3 Flow (Veh/h),Loop3 Speed (km/h),Loop4 Flow (Veh/h),Loop4 Speed (km/h),Loop5 Flow (Veh/h),Loop5 Speed (km/h),Loop6 Flow (Veh/h),Loop6 Speed (km/h),Loop7 Flow (Veh/h),Loop7 Speed (km/h),Loop8 Flow (Veh/h),Loop8 Speed (km/h),Loop9 Flow (Veh/h),Loop9 Speed (km/h),Loop10 Flow (Veh/h),Loop10 Speed (km/h)
Time,Week,,,,,,,,,,,,,,,,,,,,
8:00,6,2574,111.750000,2960,111.120003,2755,110.690002,3877,107.720001,2989,110.809998,3028,109.080002,3504,113.080002,3223,106.540001,2872,107.790001,2951,108.699997
8:05,6,2742,109.449997,2645,108.930000,2567,108.019997,3892,109.589996,2716,107.699997,2580,108.669998,3183,110.730003,2672,106.480003,2521,107.379997,2949,104.279999
8:10,6,2815,111.080002,2938,110.320000,2732,109.449997,3950,111.779999,2781,107.760002,2783,109.699997,3290,109.360001,2952,105.639999,2890,107.320000,3174,105.680000
8:15,6,2866,109.080002,3214,108.889999,3112,108.309998,4674,105.220001,3413,109.720001,3266,112.279999,3680,109.709999,3123,107.250000,2905,108.070000,3172,105.320000
8:20,6,2816,110.889999,3325,111.320000,2890,111.519997,4388,105.510002,3324,109.449997,3285,107.540001,3783,108.690002,3332,106.510002,3158,105.269997,3214,103.470001
8:25,6,3097,108.190002,3280,110.129997,3023,109.220001,4047,110.489998,3064,109.080002,3023,108.730003,3845,106.449997,3316,103.910004,3151,107.180000,3420,105.309998
8:30,6,2831,110.730003,3341,110.930000,3107,107.949997,4610,110.669998,2954,108.449997,3046,107.599998,3763,106.239998,3291,103.940002,3357,106.839996,3497,104.239998
8:35,6,3044,110.449997,3493,112.440002,3687,109.209999,4777,104.629997,3251,106.320000,3064,108.559998,3801,109.690002,3405,105.339996,3285,106.110001,3589,103.879997
8:40,6,3299,110.599998,3676,110.300003,3428,106.400002,4919,111.790001,3593,106.160004,3749,106.830002,4447,110.680000,4077,106.830002,3517,107.180000,3542,107.910004


In [7]:
result_df.to_csv('./results/submit_3.csv', index=True)

### Imputation

In [ ]:
pred = np.load('./results/20230712_005035_dm64_df32_el2_topk3_nk3_fq_h_Exp/pred.npy') # (D,L,K)

D, L, K = pred.shape
pred = pred.reshape(D * L, K) # (D*L,K)

# Define the start and end times of each range
ranges_5min = [('05:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

# Create the DatetimeIndex from the list of values
datetime_5min_test_ids_full = pd.DatetimeIndex(index_values)

pred_df = pd.DataFrame(data=pred, index=datetime_5min_test_ids_full)


# pred_time_range
pred_ranges_5min = [('08:00:00', '08:55:00'), ('12:30:00', '13:25:00'), ('17:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in pred_ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

datetime_5min_test_ids = pd.DatetimeIndex(index_values)

pred_df = pred_df.loc[datetime_5min_test_ids]

template_df = pd.read_csv('./submit_template.csv',index_col=['Time', 'Week'])

template_df.iloc[:,:] = pred_df.values

template_df.iloc[:, :10] = template_df.iloc[:, :10].astype(int)
template_df.iloc[:, 10:] = template_df.iloc[:, 10:].astype(float).applymap("{:.2f}".format)

template_df.to_csv('./results/submit.csv', index=True)